### Import Modules

In [ ]:
import pandas as pd

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [ ]:
import mlflow

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pickle

### Simple MLFlow usecase

In [ ]:
mlflow.set_tracking_uri("sqlite:///../mlruns.db")

In [ ]:
mlflow.set_experiment("NYC-Duration-Exp2")

In [ ]:
mlflow.search_experiments()

### Read and Preprocess data

In [ ]:
def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    
    return df

In [ ]:
train_data_path = '../data/yellow_tripdata_2023-01.parquet'
val_data_path = '../data/yellow_tripdata_2023-02.parquet'

In [ ]:
df_train = read_data(train_data_path)
df_val = read_data(val_data_path)

In [ ]:
# Preprocess data
str_columns = ["PULocationID", "DOLocationID"]
df_train[str_columns] = df_train[str_columns].astype('str')
df_val[str_columns] = df_val[str_columns].astype('str')

# Compute features
df_train['PU_DO'] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val['PU_DO'] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

In [ ]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID'
numerical = ['trip_distance']

# get columns as dict
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

# fit transform dict vectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

In [ ]:
X_train.shape, X_val.shape

In [ ]:
y_train = df_train.duration.values
y_val = df_val.duration.values

In [ ]:
y_train.shape, y_val.shape

### Train and evaluate model with MLFLow Tracking

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("dev", "naji")
    mlflow.log_param("train dp", train_data_path)
    mlflow.log_param("val dp", val_data_path)

    alpha = 0.5
    lr = Ridge(alpha)
    mlflow.log_param("alpha", alpha)
    
    lr.fit(X_train, y_train)
    
    y_train_pred = lr.predict(X_train)
    train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
    mlflow.log_metric("Train RMSE", train_rmse)
    
    y_val_pred = lr.predict(X_val)
    val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
    mlflow.log_metric("Val RMSE", val_rmse)

In [ ]:
print(f'Train RMSE: {train_rmse}')
print(f'Val RMSE: {val_rmse}')

### Parameter Search with XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
# Transform data to XGBoost format
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

In [ ]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

### Train best model

In [ ]:
params = {
    'learning_rate': 0.2571221506436322,
    'max_depth': 78,
    'min_child_weight': 6.7194469896916535,
    'objective': 'reg:linear',
    'reg_alpha': 0.07394910234968388,
    'reg_lambda': 0.03219643217248881,
    'seed': 42
}

In [ ]:
mlflow.xgboost.autolog(log_datasets=False)

In [ ]:
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

In [ ]:
mlflow.xgboost.autolog(disable=False)

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("model", "xgboost-best")
    mlflow.log_params(params)
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)
    mlflow.xgboost.log_model(booster, "models_mlflow")